In [1]:
import tempfile
import plotly.express as px
import pandas as pd
from lib.integrated_single_scenario import Scenario, BaseScenario
import shutil
import os

c:\Users\szvsw\repos\mit-campus-decarbonization\lib\integrated_single_scenario.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
base_ix = 0
batch = "cc9e9cff-64f0-4fbf-b17a-2f829837f896"
profiles_path = f"sdl-epengine/dev/{batch[:8]}/results"

scenarios_df = BaseScenario.make_df()

with tempfile.TemporaryDirectory() as artifacts_dir:

    # base_scenario: BaseScenario = scenarios_df.iloc[base_ix].Scenario
    base_scenario = BaseScenario(
        renovation_rate="slow",
        climate="rcp85",
        grid="decarbonization",
        retrofit="baseline",
        schedules="baseline",
        lab="baseline",
        district_system="baseline",
        nuclear="baseline",
        deep_geothermal="baseline",
        storage="baseline",
        carbon_capture="baseline"
    )
    s = Scenario(
        artifacts_dir=artifacts_dir,
        profiles_path=profiles_path,
        x=base_scenario,
    )
    s.run()


INTEGRATED:INFO: ----
INTEGRATED:INFO: 
Scenario:
	climate=rcp85
	grid=decarbonization
	retrofit=baseline
	schedules=baseline
	lab=baseline
	district_system=baseline
	nuclear=baseline
	deep_geothermal=baseline
	renovation_rate=slow
	storage=baseline
	carbon_capture=baseline
INTEGRATED:DEBUG: Fetching files from ml-for-bem/sdl-epengine/dev/cc9e9cff/results
INTEGRATED:DEBUG: Found 6144 files in sdl-epengine/dev/cc9e9cff/results
INTEGRATED:DEBUG: Found 512 source files and 512 target files
INTEGRATED:DEBUG: Downloading artifacts...


Opening source HDFs:   0%|          | 0/512 [00:00<?, ?it/s]

Opening target HDFs:   0%|          | 0/512 [00:00<?, ?it/s]

INTEGRATED:DEBUG: Finished downloading and opening artifacts.
INTEGRATED:DEBUG: Concatenating artifacts...
INTEGRATED:INFO: Finished loading artifacts.
INTEGRATED:DEBUG: Making load sequence...
INTEGRATED:DEBUG: Interpolating between 2025 and 2030 for year 2025
INTEGRATED:DEBUG: Interpolating between 2025 and 2030 for year 2026
INTEGRATED:DEBUG: Interpolating between 2025 and 2030 for year 2027
INTEGRATED:DEBUG: Interpolating between 2025 and 2030 for year 2028
INTEGRATED:DEBUG: Interpolating between 2025 and 2030 for year 2029
INTEGRATED:DEBUG: Interpolating between 2030 and 2040 for year 2030
INTEGRATED:DEBUG: Interpolating between 2030 and 2040 for year 2031
INTEGRATED:DEBUG: Interpolating between 2030 and 2040 for year 2032
INTEGRATED:DEBUG: Interpolating between 2030 and 2040 for year 2033
INTEGRATED:DEBUG: Interpolating between 2030 and 2040 for year 2034
INTEGRATED:DEBUG: Interpolating between 2030 and 2040 for year 2035
INTEGRATED:DEBUG: Interpolating between 2030 and 2040 for 

epw_year
2025    1.281267e+08
2026    1.275241e+08
2027    1.269169e+08
2028    1.263051e+08
2029    1.256885e+08
2030    1.250673e+08
2031    1.248244e+08
2032    1.245797e+08
2033    1.243333e+08
2034    1.240850e+08
2035    1.238348e+08
2036    1.236954e+08
2037    1.235552e+08
2038    1.234143e+08
2039    1.232727e+08
2040    1.231305e+08
2041    1.228399e+08
2042    1.225482e+08
2043    1.222554e+08
2044    1.219613e+08
2045    1.216662e+08
2046    1.215111e+08
2047    1.213558e+08
2048    1.212005e+08
2049    1.210451e+08
2050    1.208895e+08
dtype: float64


In [237]:
if os.path.exists(artifacts_dir):
    print("cleaning")
    shutil.rmtree(artifacts_dir)

# Plotting

In [150]:
actual_area = (
    s.source_df.reset_index("actual_area").groupby("building_id").first()["actual_area"]
)
actual_area
a = (
    s.scheduled_loads.xs(2025, level="epw_year")
    .T.groupby("end_use")
    .sum()
    .T.reset_index("segment", drop=True)
)
fig = px.histogram(
    a.melt(ignore_index=False, value_name="energy"),
    x="energy",
    facet_col="end_use",
    facet_col_wrap=2,
    height=400,
    log_y=True,
)
# fig.show()
factor = (a.Heating / a.Cooling).rename("heating to cooling ratio")
fig = px.histogram(factor, height=400, nbins=50)
# fig.show()
fig = px.scatter(x=a.Cooling, y=factor)
# fig.show()
df = pd.concat([a, factor, actual_area], axis=1)
df
# a_eui = a /actual_area.loc[a.index].values.reshape(-1,1)
# a_eui

k = s.source_df.xs(2025, level="epw_year")
# k = k.xs(0, level="lab_weight")
k = k.T.groupby(level=0).sum().T
# data = k.xs(0, level="lab_weight")
data = k

cool_eui = pd.Series(data.Cooling.values / data.reset_index().actual_area.values, index=data.index, name="Cooling EUI")
heat_eui = pd.Series(data.Heating.values / data.reset_index().actual_area.values, index=data.index, name="Heating EUI")
elec_eui = pd.Series(data.Electricity.values / data.reset_index().actual_area.values, index=data.index, name="Electricity EUI")
water_eui = pd.Series(data.Water.values / data.reset_index().actual_area.values, index=data.index, name="Water EUI")
df = pd.concat([cool_eui, heat_eui, elec_eui, water_eui], axis=1)
df = df.melt(ignore_index=False, value_name="EUI", var_name="End Use").reset_index()
for typology in df.TYPOLOGY.unique():
    fig = px.histogram(df[df.TYPOLOGY == typology], x="EUI", facet_row="TYPOLOGY", facet_col_wrap=2, nbins=100, color="End Use", barmode="group", title=f"{typology} EUI")
    fig.show()
print("-----------------")
cool_gross = pd.Series(data.Cooling.values / 8760 / 1000, index=data.index, name="Cooling")
heat_gross = pd.Series(data.Heating.values / 8760 / 1000, index=data.index, name="Heating")
elec_gross = pd.Series(data.Electricity.values / 8760 / 1000, index=data.index, name="Electricity")
water_gross = pd.Series(data.Water.values / 8760 / 1000, index=data.index, name="Water")
df = pd.concat([cool_gross, heat_gross, elec_gross, water_gross], axis=1)
df = df.melt(ignore_index=False, value_name="Power (avg MW)", var_name="End Use").reset_index()
for typology in df.TYPOLOGY.unique():
    fig = px.histogram(df[df.TYPOLOGY == typology], x="Power (avg MW)", facet_row="TYPOLOGY", facet_col_wrap=2, nbins=100, color="End Use", barmode="group", title=f"{typology} Gross")
    fig.show()



c:\tools\Anaconda3\envs\mit-campus-decarbonization\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



-----------------


In [176]:
data = s.source_df.xs(2025, level="epw_year")

average_power_draw = data.T.groupby(level=0).describe().T.xs("mean", level=-1)["Electricity"].sort_values(ascending=False)
max_power_draw = data.T.groupby(level=0).describe().T.xs("max", level=-1)["Electricity"].sort_values(ascending=False)
min_power_draw = data.T.groupby(level=0).describe().T.xs("min", level=-1)["Electricity"].sort_values(ascending=False)
average_power_draw.index = average_power_draw.reset_index("building_id")["building_id"]
(average_power_draw.rename("Average Power Draw (MW)").iloc[:20] / 1000).round(2)

((data.sum().groupby(level=0).describe().loc["Electricity"]).rename("Campus Wide Power Draw from lights and equip (MW)") / 1000).round(2) 




count     8.76
mean     31.38
std       5.17
min      26.25
25%      27.18
50%      28.30
75%      35.67
max      40.07
Name: Campus Wide Power Draw from lights and equip (MW), dtype: float64

In [60]:
df = s.scheduled_loads.groupby("epw_year").sum().T.groupby(level="end_use").sum().T
annual = pd.DataFrame(df.sum(axis=1), columns=["energy"]).reset_index()
fig = px.line(annual, x="epw_year", y="energy")
fig.show()
by_end_use = df.melt(ignore_index=False, var_name="end_use", value_name="energy").reset_index()
fig = px.line(by_end_use, x="epw_year", y="energy", color="end_use")
fig.show()



c:\tools\Anaconda3\envs\mit-campus-decarbonization\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [68]:
k = (
    s.scheduled_loads.groupby("epw_year")
    .sum()
    .stack(level="DateTime", future_stack=True)
    .xs(2025, level="epw_year")
)
k.Heating = k.Heating / 0.4
k.Cooling = k.Cooling / 3.5
k = k.melt(ignore_index=False, value_name="energy", var_name="end_use").reset_index()
px.line(
    k,
    x="DateTime",
    y="energy",
    color="end_use",
)

c:\tools\Anaconda3\envs\mit-campus-decarbonization\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [62]:
fig = px.histogram(s.scheduled_loads["Electricity"].groupby("epw_year").sum().loc[2025].values / 1000)
# set the xaxis label to "Heating Ideal Loads, hourly, MWh"
fig.update_layout(xaxis_title="Heating Ideal Loads, hourly, MWh")
fig.show()


# Phasing

In [9]:
df = pd.read_csv("data/iam/district_phased.csv")
phased_bldgs = set(df["Building Number"])
bldg_loads_bldgs = set(s.bldg_loads(2025).index.get_level_values("building_id").unique())
in_phased_not_in_bldg_loads = phased_bldgs - bldg_loads_bldgs
in_bldg_loads_not_in_phased = bldg_loads_bldgs - phased_bldgs
print("In phased but not in building loads:", in_phased_not_in_bldg_loads)
print("In building loads but not in phased:", in_bldg_loads_not_in_phased)





INTEGRATED:DEBUG: Interpolating between 2025 and 2030 for year 2025


In phased but not in building loads: {'W88', 'W87', 'W18', 'W51c'}
In building loads but not in phased: {'NW38A', 'W85FG', 'NW36', 'NW12', 'W85', 'NW61', 'N16B', 'W85DE', 'W2', '6', 'W83', 'W5', 'E37', 'NW38', '42C', 'W51C', 'N52', 'N57', 'NW12A', 'W85HJK', '62', 'N10', 'WW15', 'N16C', '6C', 'N9', 'N16A', 'W41', '45', 'N51', 'W51D', '7A', 'NW30', '7', 'W85ABC', 'W20', 'W8', '64', 'W4', 'W46', 'NW20'}
